## PCX - Phuture Crypto Market Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = None

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[['bitcoin'],['wrapped-bitcoin']],
    remove_category_assets=['staked-ether','wrapped-bitcoin','rocket-pool-eth','frax-ether','staked-frax-ether']
    )
liquidity

Note: mantle, marketcap data available only for 172 < 180 days
Excluding beam-2, pricing data available only for 79 < 180 days
0x0000000000000000000000000000000000001010 polygon-pos
0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
0x925206b8a707096ed26ae47c84747fe0bb734f59 ethereum
0x19de6b897ed14a376dda0fe53a5420d2ac828a28 ethereum
0x7205705771547cf79201111b4bd8aaf29467b9ec polygon-pos
 ethereum
0x44c784266cf024a60e8acf2427b9857ace194c5d avalanche
0x23ee2343b892b1bb63503a4fabc840e0e2c6810f optimistic-ethereum
0x23ee2343b892b1bb63503a4fabc840e0e2c6810f base
0x8b1f4432f943c465a973fedc6d7aa50fc96f1f65 fantom
0xa0b862f60edef4452f25b4160f177db44deb6cf1 arbitrum-one
0x7c598c96d02398d89fbcb9d41eab3df0c16f227d fantom


,spot price,delivery price,slippage,blockchain
id,,,,
lido-dao,0.273232,0.273815,0.002133,ethereum
maker,0.000472,0.000472,0.000445,ethereum
wrapped-bitcoin,0.000022,0.000022,0.000000,arbitrum-one
ethereum,0.000381,0.000381,-0.000465,ethereum
woo-network,2.223535,2.221763,-0.000797,ethereum
binancecoin,0.003226,0.003222,-0.001033,binance-smart-chain
pancakeswap-token,0.328439,0.327837,-0.001834,binance-smart-chain
matic-network,1.086571,1.084321,-0.002071,polygon-pos
gmx,0.017805,0.017764,-0.002324,arbitrum-one


## Show Results

In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
wrapped-bitcoin,BTC,903595459480,0.314262,20596,0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f,arbitrum-one
ethereum,ETH,314900253325,0.314262,20596,ETH,ethereum
binancecoin,BNB,47695819851,0.149183,9777,BNB,binance-smart-chain
avalanche-2,AVAX,14188656946,0.044379,2909,AVAX,avalanche
matic-network,MATIC,8507115834,0.026609,1744,0x0000000000000000000000000000000000001010,polygon-pos
chainlink,LINK,8438258160,0.026393,1730,0x514910771af9ca656af840dff83e8264ecf986ca,ethereum
the-open-network,TON,7615809954,0.023821,1562,0x582d872a1b094fc48f5de31d3b73f2d9be47def1,ethereum
shiba-inu,SHIB,5996869378,0.018757,1230,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,ethereum
uniswap,UNI,5109972245,0.015983,1048,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
